## **PySpark SQL - Getting Started Notebook**
This notebook provides an example for installing spark dependencies and a simple "getting started" syntax for PySpark SQL.

NOTE: This notebook is designed to be used with Google's Colab notebook and the Python 3 runtime.

## **Install Spark Dependencies**



In [ ]:
# Install Spark dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!rm spark-3.1.1-bin-hadoop3.2.tgz
!wget --no-cookies --no-check-certificate https://apache.claz.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar zxvf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark

In [11]:
!ls -al | grep spark

drwxr-xr-x 13 1000 1000      4096 Feb 22 02:11 spark-3.1.1-bin-hadoop3.2
-rw-r--r--  1 root root 228721937 Feb 22 02:45 spark-3.1.1-bin-hadoop3.2.tgz


## **Set env variables within Pyspark**

In [12]:
# Set up required environment variables
import os
os.environ["JAVA_HOME"]  = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

## **Download Data**

In [13]:
!wget https://raw.githubusercontent.com/zaratsian/iaa_2021/main/session_02/bikeshare_station_info.csv
!wget https://raw.githubusercontent.com/zaratsian/iaa_2021/main/session_02/bikeshare_trips.csv

--2021-03-20 13:32:00--  https://raw.githubusercontent.com/zaratsian/iaa_2021/main/session_02/bikeshare_station_info.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80415 (79K) [text/plain]
Saving to: ‘bikeshare_station_info.csv.1’

bikeshare_station_i 100%[===================>]  78.53K  --.-KB/s    in 0.009s  

2021-03-20 13:32:00 (8.65 MB/s) - ‘bikeshare_station_info.csv.1’ saved [80415/80415]

--2021-03-20 13:32:00--  https://raw.githubusercontent.com/zaratsian/iaa_2021/main/session_02/bikeshare_trips.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiti

## **Import Python and PySpark Libraries**

In [14]:
import datetime
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import monotonically_increasing_id, col, expr, when, concat, lit, udf, split
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import GBTRegressor, LinearRegression, GeneralizedLinearRegression
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import VectorIndexer, VectorAssembler, StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

## **Initialize Spark Session**

In [15]:
spark = SparkSession.builder.appName("Bikesharing SparkSQL").master("local[*]").getOrCreate()

## **Read CSV into Spark**

In [16]:
bikeshare_trips = spark.read.load('bikeshare_trips.csv', format="csv", header=True, inferSchema=True)

## **Display first few records**

In [17]:
bikeshare_trips.show(5, truncate=False)

+--------------------+------------+-----------------------+-----------------------+----------------+-----------------------+--------------------+--------------+-----------+--------+---------------+-------------------+----------------------+-----------------------+--------------------+---------------------+-----------------+-------------+-----------------------+------------------------------+----------------------------+
|trip_id             |duration_sec|start_date             |start_station_name     |start_station_id|end_date               |end_station_name    |end_station_id|bike_number|zip_code|subscriber_type|c_subscription_type|start_station_latitude|start_station_longitude|end_station_latitude|end_station_longitude|member_birth_year|member_gender|bike_share_for_all_trip|start_station_geom            |end_station_geom            |
+--------------------+------------+-----------------------+-----------------------+----------------+-----------------------+--------------------+-------

## **Execute Sample SparkSQL query**

In [18]:
bikeshare_trips.createOrReplaceTempView("bikeshare_trips")
spark.sql("SELECT subscriber_type, count(*) as count FROM bikeshare_trips group by subscriber_type order by count desc").show(truncate=False)

+---------------+-----+
|subscriber_type|count|
+---------------+-----+
|Subscriber     |13685|
|Customer       |2315 |
+---------------+-----+

